# Dask Run

In [1]:
import os
from dask.distributed import Client
import dask.dataframe as dd
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [2]:
# PATH_DATA =  '../../data-testing/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output3/'

SLA_NAME = 'dask-sla.pkl'
MEM_NAME = 'dask-mem.pkl'


In [3]:
all_sla = {}
all_memory = {}

In [4]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

In [5]:
# all_sla = joblib.load(PATH_OUTPUT+SLA_NAME)
# all_memory = joblib.load(PATH_OUTPUT+MEM_NAME)

In [6]:
client = Client(n_workers=12)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 24,Total memory: 23.42 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60049,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 23.42 GiB
Comm: tcp://127.0.0.1:60107,Total threads: 2
Dashboard: http://127.0.0.1:60124/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:60052,


##     Task1 & Task2- I/O

In [7]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}


all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [8]:
for i in range(4):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start_mem = get_memory()
        start = time.perf_counter()
        dtemp1 = dd.read_csv(PATH_DATA+fn).compute()
        end = time.perf_counter() - start
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start_mem = get_memory()
        start = time.perf_counter()
        dtemp1.to_csv(TEMP_PATH + 'temp.csv',index=False)
        end = time.perf_counter() - start
        end_mem = get_memory() - start_mem
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start_mem = get_memory()
    start = time.perf_counter()
    dtemp1 = dd.read_parquet(PATH_DATA+fn, engine='pyarrow').compute()
    end = time.perf_counter() - start
    end_mem = get_memory() - start_mem
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    
    start_mem = get_memory()
    start = time.perf_counter()
    dtemp1.to_parquet(TEMP_PATH + 'temp.parquet',index=False)
    end = time.perf_counter() - start
    end_mem = get_memory() - start_mem
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(2)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1
2
3


In [9]:
all_sla

{'task1': {'csv': {'0': 0.9505215000000007,
   '1': 2.1016592000000003,
   '5': 5.543132299999996,
   '10': 10.163982500000003},
  'prq': {'0': 0.5233458000000013,
   '1': 1.4320863000000017,
   '5': 5.732085600000005,
   '10': 8.112662600000021}},
 'task2': {'csv': {'0': 0.5509322999999995,
   '1': 4.507115299999999,
   '5': 21.600442700000002,
   '10': 44.6360734},
  'prq': {'0': 0.10301979999999844,
   '1': 0.6555947000000018,
   '5': 2.891763400000002,
   '10': 6.067186499999991}}}

In [10]:
all_memory

{'task1': {'csv': {'0': 0.017379328000000527,
   '1': 0.261705727999999,
   '5': 0.6904872960000006,
   '10': 1.3822197759999995},
  'prq': {'0': 0.06719488000000062,
   '1': 0.20419788800000127,
   '5': 0.568856576,
   '10': 0.9189253119999989}},
 'task2': {'csv': {'0': -0.010293247999999977,
   '1': -0.050049023999999775,
   '5': -0.07478476799999889,
   '10': -0.033136640000000384},
  'prq': {'0': 0.004960256000000385,
   '1': -0.0028467200000008575,
   '5': -0.14498201600000016,
   '10': -0.10193715199999964}}}

In [10]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [10]:
all_df = [dd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [2,3,4,5,6,7]]
# all_df = [dd.read_parquet(PATH_DATA+'data_{}.parquet'.format(i)) for i in [1,2,3]]
df_right = dd.read_parquet(PATH_DATA+'data_to_join.parquet')

In [11]:
len(all_df)

3

In [13]:
# type(df_right)

## Task3 - Sorting

In [12]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [13]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=[True,False]).compute()
    dtemp2 = dtemp1.sort_values(['Date','Amount'],ascending=True).compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


1.03
5.03
10.06


In [14]:
all_sla[n_task]

{'1': 2.1536146999999914, '5': 6.944418799999994, '10': 32.05392259999999}

In [15]:
all_memory[n_task]

{'1': 0.20388659199999992, '5': 0.5947883520000001, '10': 1.8073927679999997}

## Task4 - Filtering

In [ ]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [ ]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))].compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


In [ ]:
all_sla[n_task]

In [ ]:
all_memory[n_task]

## Task5 - Merging

In [16]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [17]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dd.merge(dtemp1,df_right,on=['Date','ship-service-level'],how='left').compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03
10.06


In [18]:
all_sla[n_task]

{'1': 2.094254700000022, '5': 6.993969400000026, '10': 10.731813200000033}

In [19]:
all_memory[n_task]

{'1': 0.13430374400000034, '5': 0.43998822400000037, '10': 0.6472417280000009}

## Task6 - udf apply

In [ ]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [ ]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [ ]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp1['Amount2'] = dtemp1['Amount'].apply(fun).compute()
    dtemp1 = dtemp1.assign(Amount2 = dtemp1['Amount'].apply(fun, meta=('Amount', 'float64'))).compute()
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

In [ ]:
all_sla[n_task]

In [ ]:
all_memory[n_task]

## Task7 - aggregation

In [ ]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [ ]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [ ]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    # dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':[np.mean, np.size, p25, p75]})
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'}).compute()

    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0].compute()/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

In [ ]:
all_sla[n_task]

In [ ]:
all_memory[n_task]

## Save results

In [43]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output2/dask-mem.pkl']

In [41]:
all_sla.keys()

dict_keys(['task1', 'task2', 'task3', 'task4', 'task5', 'task6', 'task7'])

In [42]:
all_sla

{'task1': {'csv': {'0': 0.31273062600007506,
   '1': 1.1573291619999964,
   '5': 4.665960830000131,
   '10': 9.092289742000048,
   '25': 22.510686222000004,
   '50': 44.5934548419998},
  'prq': {'0': 0.1093772040001113,
   '1': 0.4811294030000681,
   '5': 2.1318858540000747,
   '10': 3.998179236999931,
   '25': 8.53980606999994,
   '50': 16.647448817000168,
   '75': 24.425851702999807,
   '100': 33.285870936000265,
   '130': 42.347066524000184,
   '175': 58.4511481440004}},
 'task2': {'csv': {'0': 0.3765234020002026,
   '1': 2.944178162000071,
   '5': 14.416140815000063,
   '10': 28.891955409000047,
   '25': 72.18523263299994,
   '50': 145.38576077899984},
  'prq': {'0': 0.08917017499993563,
   '1': 0.6809641170000305,
   '5': 3.0853535629998987,
   '10': 6.333279904999927,
   '25': 16.11656594100009,
   '50': 30.999546537000242,
   '75': 40.27982504300007,
   '100': 57.8371491490002,
   '130': 76.49357919600016,
   '175': 99.3272646989999}},
 'task3': {'25': 103.93448567799987,
  '50'